In [5]:
import mlflow
from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri("http://localhost:5050")
client = MlflowClient()
model_name = "dtc_persona_clustering_model"

# You've already found the latest version, let's say its version number is '19'
latest_version_info = client.get_latest_versions(name=model_name, stages=["None"])[0]
version_to_promote = latest_version_info.version

print(f"Promoting version {version_to_promote} of model '{model_name}' to 'Production'.")

# This is the key step:
client.transition_model_version_stage(
    name=model_name,
    version=version_to_promote,
    stage="Production",
    archive_existing_versions=True # This will move any existing 'Production' model to 'Archived'
)

print("Promotion successful.")

Promoting version 19 of model 'dtc_persona_clustering_model' to 'Production'.
Promotion successful.


/var/folders/2p/knkmky2x69s7jl6b1s6y8s3r0000gn/T/ipykernel_46089/3804978902.py:9: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_version_info = client.get_latest_versions(name=model_name, stages=["None"])[0]
/var/folders/2p/knkmky2x69s7jl6b1s6y8s3r0000gn/T/ipykernel_46089/3804978902.py:15: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [6]:
import mlflow


# Set the tracking URI to point to your MLflow server
mlflow.set_tracking_uri("http://localhost:5050")

MODEL_NAME = "dtc_persona_clustering_model"
MODEL_STAGE = "Production"
MODEL_URI = f"models:/{MODEL_NAME}/{MODEL_STAGE}"

print(f"Loading model from: {MODEL_URI}")
# MLflow will automatically resolve this to the correct GCS path and download it.
loaded_model = mlflow.pyfunc.load_model(MODEL_URI)
print("Model loaded successfully.")


Loading model from: models:/dtc_persona_clustering_model/Production


2025/07/22 09:47:06 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - lz4 (current: uninstalled, required: lz4==4.4.4)
 - pandas (current: 2.2.3, required: pandas==1.5.3)
 - psutil (current: 7.0.0, required: psutil==5.9.8)
 - scipy (current: 1.13.1, required: scipy==1.15.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/07/22 09:47:06 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.10.17`, differs from the version of Python that is currently running, `Python 3.9.22`, and may be incompatible


Model loaded successfully.


In [7]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 265157f127dd427a8355e882381df322